<a href="https://colab.research.google.com/github/arfafax/StyleGAN2_experiments/blob/master/TADNE_Editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Anime Does Not Exist - Ganspace Editor 


### To run:
**Click "Open in Playground" above (if you see that option) and then Runtime > Run All and wait a few minutes for everything to load. (Takes about 4 minutes)**


---


### Credits

Customizable anime images using [aydao](https://twitter.com/aydaoAI)'s anime StyleGAN2 model from [TADNE](https://thisanimedoesnotexist.ai/).

Latent directions were discovered using [@harskish](https://twitter.com/harskish)'s [Ganspace](https://github.com/harskish/ganspace) [notebook](https://colab.research.google.com/github/harskish/ganspace/blob/master/notebooks/Ganspace_colab.ipynb) put together by [@realmeatyhuman](https://twitter.com/realmeatyhuman).


Notebook by [arfa](https://twitter.com/arfafax)


<div>
<a href="https://www.twitter.com/arfafax">
<img src="https://thisfursonadoesnotexist.com/arfa_sig.png" width="350"/>
</a>
</div>
<div>


## <- Click the triangle to view the code while you wait for it to load


In [25]:
!git clone https://github.com/shawwn/stylegan2 -b estimator /content/stylegan2

fatal: destination path '/content/stylegan2' already exists and is not an empty directory.


In [26]:
import gdown
gdown.download('https://drive.google.com/uc?id=1qNhyusI0hwBLI-HOavkNP5I0J0-kcN4C', 'network-tadne.pkl', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1qNhyusI0hwBLI-HOavkNP5I0J0-kcN4C
To: /content/animations/network-tadne.pkl
1.06GB [00:06, 171MB/s]


'network-tadne.pkl'

In [27]:
!wget  -O /content/tadne-directions.zip https://www.thisponydoesnotexist.net/tadne-directions.zip

--2021-01-28 07:37:41--  https://www.thisponydoesnotexist.net/tadne-directions.zip
Resolving www.thisponydoesnotexist.net (www.thisponydoesnotexist.net)... 172.67.145.125, 104.21.65.129, 2606:4700:3037::ac43:917d, ...
Connecting to www.thisponydoesnotexist.net (www.thisponydoesnotexist.net)|172.67.145.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://thisponydoesnotexist.net/tadne-directions.zip [following]
--2021-01-28 07:37:41--  https://thisponydoesnotexist.net/tadne-directions.zip
Resolving thisponydoesnotexist.net (thisponydoesnotexist.net)... 172.67.145.125, 104.21.65.129, 2606:4700:3032::6815:4181, ...
Connecting to thisponydoesnotexist.net (thisponydoesnotexist.net)|172.67.145.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1966529 (1.9M) [application/zip]
Saving to: ‘/content/tadne-directions.zip’

/content/tadne-dire 100%[===================>]   1.88M  2.32MB/s    in 0.8s    

2021-01-28 07:37:

In [28]:
%tensorflow_version 1.x
%cd /content/stylegan2

/content/stylegan2


In [29]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import tensorflow as tf
import tflex

In [30]:
if 'COLAB_TPU_ADDR' in os.environ:
    os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    os.environ['NOISY'] = '1'

In [31]:
tflib.init_tf()
sess = tf.get_default_session()
sess.list_devices()
cores = tflex.get_cores()
tflex.set_override_cores(cores)
_G, _D, Gs = pickle.load(open("/content/network-tadne.pkl", "rb"))
# _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
# _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
# Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

In [32]:
 def generate_image_from_w(w, truncation_psi):
    with tflex.device('/gpu:0'):
        noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        Gs_kwargs.randomize_noise = False
        if truncation_psi is not None:
            Gs_kwargs.truncation_psi = truncation_psi
        synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation_psi, minibatch_size=1)
        images = Gs.components.synthesis.run(w, randomize_noise=False, **synthesis_kwargs)
        display(PIL.Image.fromarray(images[0], 'RGB').resize((500,500),Image.LANCZOS))

In [33]:
%cd "/content"

/content


In [ ]:
%mkdir directions
%cd directions
!unzip /content/tadne-directions.zip

mkdir: cannot create directory ‘directions’: File exists
/content/directions
Archive:  /content/tadne-directions.zip
replace c47.2_layers_6_9.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!rm /content/directions/StyleGAN2-Light_direction-ffhq-ipca-w-style-comp15-range8-9.pkl
#!rm -r /content/directions/.ipynb_checkpoints
#!mv /content/directions/directions/* /content/ganspace/directions
#!rm -r /content/directions/directions

In [ ]:
import os
named_directions = {}
latent_dirs = []
starts = []
ends = []
# 
path_to_directions = "/content/directions"

# This loads the directions in a dictionary in this format:
# {'name' : [direction_num, start, end]}
# and you load the direction by:
# directions[direction_num]

for i,file in enumerate(sorted(os.listdir(path_to_directions))):
    np_file = np.load(f'{path_to_directions}/{file}', allow_pickle=True)
    name = file.split("_layers_")[0].split("/")[-1]
    file = file.split('_')
    comp = int(name.split(".")[0][1:])
    
    named_directions[f'{name}'] = [comp, int(file[-2]), int(file[-1].split('.')[0]), f'{name}']
    if int(file[-2]) == 0:
        latent_dirs.append(np_file)


In [ ]:
named_directions

In [ ]:
import yaml
%mkdir /content/animations
%cd /content/animations

# UI

In [ ]:
#@title Run UI (make sure you've done Runtime > Run All first or it won't work)
from ipywidgets import fixed
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
from IPython.display import clear_output
from ipywidgets import Layout, Button, Box, VBox, Label

box_layout = Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='',
                    height="500px")

loaded_w = None
def display_sample(seed, truncation, direction, distance, scale, start, end, update, disp=True, save=None, noise_spec=None, **args):
    global loaded_w
    if update == False:
        print("False")
    # blockPrint()
    rng = np.random.RandomState(seed)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])
    #z = np.load("/content/drive/My Drive/latents/1006-0-0-9.npy")
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]
    if loaded_w is not None:
      all_w = loaded_w.copy()
    
    for i, item in enumerate(args):
        value = args[item]
        start_l = named_directions[item][1]
        end_l = min(16, named_directions[item][2])
        direction_l = latent_dirs[named_directions[item][0]]
        for l in range(start_l, end_l):
            all_w[0][l] = all_w[0][l] + direction_l * value * scale

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]

    generate_image_from_w(all_w, truncation)

seed = np.random.randint(0,100000)
style = {'description_width': 'initial', 'width': '150px'}
row_length = 6

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.4, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='end layer: ', continuous_update=False)


update = widgets.Checkbox(value=True, description="update")

directions_list = []
params = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}

for i, item in enumerate(named_directions):
    name = named_directions[item][3]
    widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
    directions_list.append(widget)
    params[item] = widget

top_box = widgets.HBox([seed, truncation])
rename_dropdown = widgets.Dropdown(options=named_directions.keys())
rename_box = widgets.Text()
rename = widgets.Button(description="Rename")
bot_box = widgets.HBox([rename_dropdown, rename_box, rename])

ui = widgets.VBox([top_box, bot_box])

grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
for i in range(len(directions_list)//6):
    for j in range(6):
        grid[i, j] = directions_list[6*i+j]
    
ui2 = grid


random = widgets.Button(description="Randomize Sliders")
reset = widgets.Button(description="Reset Sliders")
mobile = widgets.Button(description="Mobile Mode")
desktop = widgets.Button(description="Desktop Mode")
def reset_sliders(b):
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    last_button = mobile
    if row_length == 1:
        last_button = desktop
    display(ui, out, ui2, reset, random, last_button, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def random_sliders(b):
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=np.random.normal(scale=2.5), description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    last_button = mobile
    if row_length == 1:
        last_button = desktop
    display(ui, out, ui2, reset, random, last_button, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def mobile_mode(b):
    global row_length
    global params
    row_length = 1
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=params[item].value, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    row_list = []
    foo = []
    for i, item in enumerate(directions_list):
        row_list.append(item)
        if len(row_list) == row_length:
            bar = widgets.HBox(row_list)
            foo.append(bar)
            row_list = []
    bar = widgets.HBox(row_list)
    foo.append(bar)

    ui2 = widgets.VBox(foo)
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    display(ui, out, ui2, reset, random, desktop, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def desktop_mode(b):
    global row_length
    global params
    row_length = 6
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=params[item].value, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    display(ui, out, ui2, reset, random, mobile, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def rename_slider(b):
    global row_length
    global params
    print(rename_dropdown.value, rename_box.value)
    named_directions[rename_dropdown.value][3] = rename_box.value
    #del named_directions[rename_dropdown.value]
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        value_new = 0
        if item in params:
            value_new = params[item].value
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=value_new, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    display(ui, out, ui2, reset, random, mobile, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))
  
save_name = widgets.Text()
load_name = widgets.Text()
save_btn = widgets.Button(description="Save")
load_btn = widgets.Button(description="Load")

def load_w(b):
    global loaded_w
    print("Loading ", load_name.value + ".npy")
    loaded_w = np.load(load_name.value + ".npy")

def save_w(b):
    global loaded_w
    rng = np.random.RandomState(params['seed'].value)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]
    if loaded_w is not None:
        all_w = loaded_w.copy()

    for i, item in enumerate(params):
        if item not in ['seed', 'truncation', 'direction', 'distance', 'scale', 'start', 'end', 'update']:
            value = params[item].value
            start_l = named_directions[item][1]
            end_l = min(16, named_directions[item][2])
            direction_l = latent_dirs[named_directions[item][0]]
            for l in range(start_l, end_l):
                all_w[0][l] = all_w[0][l] + direction_l * value * params['scale'].value

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * params['truncation'].value # [minibatch, layer, component]

    from datetime import datetime
    now = datetime.now()

    out_name = now.isoformat()
    
    if save_name.value:
        out_name = save_name.value
    np.save(out_name + ".npy", all_w)
    print("saved ", out_name + ".npy")

random.on_click(random_sliders)
reset.on_click(reset_sliders)
mobile.on_click(mobile_mode)
desktop.on_click(desktop_mode)
rename.on_click(rename_slider)
save_btn.on_click(save_w)
load_btn.on_click(load_w)
out = widgets.interactive_output(display_sample, params)
display(ui, out, ui2, reset, random, mobile, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))



# Animation stuff

In [ ]:
import math
from PIL import ImageFont
from PIL import ImageDraw
def interpolate_between_ws(seed_array, truncation=0.5, duration_sec = 12.0, smoothing_sec = 1.0, mp4_fps = 20, filename=None, text=False):
    #_G, _D, Gs = pickle.load(open("/content/network-e621.pkl", "rb"))
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation, minibatch_size=8)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    #if seed_array[0] != seed_array[-1]:
    #    seed_array.append(seed_array[0])
    

    all_w = seed_array
        
    num_frames = int(np.rint(duration_sec * mp4_fps))
        
    def make_frame(t):
        blend = ((len(seed_array)-1)*t/duration_sec)%1.0
        src_i = math.floor((t/duration_sec)*(len(seed_array)-1))
        dst_i = src_i + 1
        #print(t, blend, src_i, dst_i)
        all_w_new = (blend * all_w[dst_i]) + (1 - blend) * all_w[src_i]
        all_images_src = Gs.components.synthesis.run(all_w_new, randomize_noise=False, **synthesis_kwargs)
        #all_images_dst = Gs.components.synthesis.run(all_w_dst, randomize_noise=False, **synthesis_kwargs)
        if text:
            new_im = PIL.Image.new('RGB', (512, 600))
            new_im.paste(PIL.Image.fromarray(np.median(all_images_src, axis=0).astype(np.uint8)), (0, 0))
            draw = ImageDraw.Draw(new_im)
            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", size=16)
            draw.text((10, 512), "{:0.2f}".format((1-blend)), (255, 0, 0), font=font)
            draw.text((50, 512), str(seed_array[src_i]), (255, 255, 255), font=font)
            draw.text((10, 550), "{:0.2f}".format((blend)), (0, 255, 0), font=font)
            draw.text((50, 550), str(seed_array[dst_i]), (255, 255, 255), font=font)
            return np.array(new_im)
        else:
            return all_images_src[0]

    
    import moviepy.editor
    from datetime import datetime
    now = datetime.now()

    np.save(now.isoformat() + ".npy", all_w)
    mp4_file = '%s.mp4' % (now)
    if filename:
        mp4_file = filename
    mp4_codec = 'libx264'
    mp4_bitrate = '5M'

    video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
    video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)
    
    return mp4_file

In [ ]:
import glob
def preview_files():
    for file in sorted(glob.glob("*.npy")):
      try:
        truncation = 1
        print(file)
        w = np.load(file)
        noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.randomize_noise = False


        if truncation != 1:
            w_avg = Gs.get_var('dlatent_avg')
            all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]

        generate_image_from_w(w, truncation)
      except:
        print("Error")

# Generate animation

In [ ]:
!error Causing an error so the following cells are not run automatically....

Run this cell to preview all of the snapshots you've saved:

In [ ]:
preview_files()

List all of the files you want to animate between:

In [ ]:
ws = '''
pose1.npy
pose2.npy
pose3.npy
'''.split()
ws


Run this to generate the animation:

In [ ]:
w_arr = []
for w in ws:
    w_val = np.load(w)
    w_arr.append(w_val)

output_file = interpolate_between_ws(w_arr, duration_sec = 2.0, smoothing_sec = 1.0, mp4_fps = 30)

Run this to download the animation:

In [ ]:
from google.colab import files
files.download(output_file) 